In [2]:
import pandas as pd

# Replace 'your_file.xlsx' with the path to your Excel file
file_path = '../../storage/Judicial/raw_data_juicios.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)

# print head
print(df.head())

        Cédula        id_Juicio                                  Delito  \
0  100004407.0  03204201700015               COBRO DE PAGARÉ A LA ORDEN   
1  100007269.0  01204201905175                                ALIMENTOS   
2  100007269.0  01204201903452                                ALIMENTOS   
3  100007269.0  01204201900368   INVESTIGACIÓN DE PATERNIDAD/MATERNIDAD   
4  100013465.0  01371202300415                PAGO DE HABERES LABORALES   

         Fecha_de_Ingreso        mes     año                 Estado  \
0 2017-01-09 13:23:24.813    January  2017.0              No activo   
1 2019-09-09 15:50:33.447  September  2019.0  No se pudo determinar   
2 2019-06-12 13:19:18.733       June  2019.0              No activo   
3 2019-01-18 21:50:53.770    January  2019.0              No activo   
4 2023-12-05 17:28:11.537   December  2023.0                 Activo   

  Monto_por_juicio  
0              NaN  
1              NaN  
2              NaN  
3              NaN  
4              Na

#### Fist wee need to clasify the juicios types into 5 diffrent catgories: Alimentos, Familiares, Penales, Estafas, Cobros

In [44]:
# get all difrent types of value sin the colum 'Delito'
unique_delitos = df['Delito'].unique()
print(unique_delitos)
print(unique_delitos.shape[0])

['COBRO DE PAGARÉ A LA ORDEN' 'ALIMENTOS'
 'INVESTIGACIÓN DE PATERNIDAD/MATERNIDAD' 'PAGO DE HABERES LABORALES'
 'PRESCRIPCIÓN ADQUISITIVA DE DOMINIO'
 'PRESCRIPCIÓN EXTRAORDINARIA ADQUISITIVA DE DOMINIO' 'LESIONES'
 'AMPARO POSESORIO' 'DERECHOS FUNDAMENTALES DEL CONSUMIDOR'
 'ARCHIVO DE LA INVESTIGACIÓN PREVIA'
 'OPOSICIÓN A LA RENDICIÓN DE CUENTAS' 'COBRO DE DINERO' 'REINVINDICACIÓN'
 'RESOLUCIÓN DE CONTRATO'
 'COBRO DE FACTURAS POR BIENES Y SERVICIOS, HONORARIOS PROFESIONALES CUANDO LA PRETENSIÓN NO SEA EXIGIBLE EN PROCEDIMIENTO MONITORIO O EN VIA EJECUTIVA'
 'TERMINACIÓN DE CONTRATO' 'COBRO DE LETRA DE CAMBIO'
 'CONTRAVENCIONES DE TRÁNSITO' 'CONTRATO DE MUTUO O PRESTAMO'
 'DIVORCIO POR CAUSAL'
 'INCUMPLIMIENTO DE DECISIONES LEGÍTIMAS DE AUTORIDAD COMPETENTE'
 'COBRO DE HONORARIOS DE ABOGADO' 'PAGO POR CONSIGNACIÓN'
 'FACTURAS O DOCUMENTOS ART. 356 NUM.2' 'FACTURA O PRIMA DE SEGUROS'
 'PARTICION NO VOLUNTARIA' 'APLICACIÓN DEL PRINCIPIO DE OPORTUNIDAD'
 'NULIDAD DEL ACTA DE MEDIACIÓN

In [85]:

def filter_delitos(delitos, key_words):
    filted_delitos = []
    remaing_delitos = []
    for delito in delitos:
        # if delitos is a string 
        if type(delito) == str:
            # for every worrd in delitos check if it is in key_words_with_familiares
            for key_word in key_words:
                if key_word in delito:
                    # add delitos to filted_delitos
                    filted_delitos.append(delito)
                    break
            # if delitos is not in family_delitos add it to filtered_delitos
            if delito not in filted_delitos:
                remaing_delitos.append(delito)
    # return the filtered delitos and the delitos that are not in the key_words
    return filted_delitos, remaing_delitos

In [89]:
categories = {

'alimentos' : ['ALIMENTOS', 'PENSIÓN'],

'familiares' : ['DIVORCIO','UNIÓN DE HECHO', 'GUARDA', 'CUSTODIA', 'VISITA', 'PATRIA POTESTAD', 'ADOPCIÓN', 
    'FILIACIÓN', 'PATERNIDAD', 'MATERNIDAD', 'PATERNIDAD/MATERNIDAD', 'FAMILIAR', 'FAMILIA', 'CONYUGAL', 
    'CONCUBINATO', 'CONCUBINO', 'CONCUBINA', 'MATRIMONIO', 'CONYUGE', 'CONYUGES','HIJO', 'HIJA',
    'HERENCIA', 'UNIÓN DE HECHO',
    ],

'penales' : ['PENAL', 'PENALES', 'DELITO', 'DELITOS', 'CRIMEN', 'CRIMINAL', 'CRIMINALES', 'CRIMINALIDAD', 'SECUESTRO',
    'HOMICIDIO', 'ROBO', 'VIOLACIÓN', 'VIOLENCIA', 'VIOLENCIA DE GÉNERO', 'VIOLENCIA DE GENERO', 'VIOLENCIA DE PAREJA',
    'VIOLENCIA DE PAREJAS', 'VIOLENCIA DE FAMILIA', 'VIOLENCIA FAMILIAR', 'VIOLENCIA INTRAFAMILIAR', 'VIOLENCIA INTRA FAMILIAR',
    'ARMAS', 'DROGAS', 'NARCOTRAFICO', 'NARCOTRÁFICO', 'NARCOTRAFICANTE', 'NARCOTRAFICANTES', 'NARCOTRAFICANTE',
    'LESIONES', 'COHECHO', 'VIOLENTO', 'ABIGEATO',  
    'ODIO', 'DISCRIMINACIÓN', 'DISCRIMINACION', 'DISCRIMINATORIO', 'DISCRIMINATORIA', 'DISCRIMINATORIOS', 'DISCRIMINATORIAS',
    'ACOSO', 'ACOSO SEXUAL', 'ACOSO LABORAL', 'ACOSO ESCOLAR', 'ACOSO CIBERNÉTICO', 'ACOSO CIBERNETICO', 'ACOSO POLÍTICO', 'ACOSO POLÍTICO',
    'ASALTO', 'ASALTOS', 'ASALTANTE', 'ASALTANTES', 'ASALTAR', 'ASALTANDO', 'ASALTÓ', 'ASALTADA', 'ASALTADO', 'ASALTADOS', 'ASALTADAS',
    'ASESINATO', 'ASESINATOS', 'ASESINO', 'ASESINOS', 'ASESINA', 'ASESINAS', 'ASESINAR', 'ASESINANDO', 'ASESINÓ', 'ASESINADA', 'ASESINADO', 'ASESINADOS', 'ASESINADAS',
    'EXTORSIÓN', 'EXTORSION', 'EXTORSIONES', 'EXTORSIONADOR', 'EXTORSIONADORES', 'EXTORSIONADORA', 'EXTORSIONADORAS', 'EXTORSIONADO', 'EXTORSIONADA', 'EXTORSIONADOS', 'EXTORSIONADAS',
    'CALUMNIA', 'APROPIACIÓN', 'MALTRATO', 'MUERTE', 'DAÑOS', 'EXHIBICIÓN DE DOCUMENTOS',
    ],

'estafa' : [
    'FALSIFICACIÓN', 'ESTAFA', 'ESTAFADOR', 'ESTAFADORES', 'ESTAFADORA', 'ESTAFADORAS', 'ESTAFA', 'ESTAFAS', 'ESTAFA',
    'COECHO', 'CUENTAS', 'BANCOS', 'BANCO', 'FINANCIERO', 'FINANCIEROS', 'FINANCIERA', 'FINANCIERAS', 'FINANZAS', 
    ],

'cobros' : [
 'COBRO', 'COBROS' , 'COBRANZA', 'COBRANZAS', 'COBRAR', 'COBRANDO', 'COBRÓ', 'COBRADO', 'COBRADA', 'COBRADOS', 'COBRADAS', 'PAGO', 
    'PAGOS', 'PAGAR', 'PAGANDO', 'PAGÓ', 'PAGADA', 'PAGADO', 'PAGADOS', 'PAGADAS', 'DEUDA', 'DEUDAS', 'DEUDOR', 'DEUDORES', 'DEUDORA',
    'DOMINIO', 
    ],

'propiedades' : [
    'PROPIEDAD', 'PROPIEDADES', 'INMUEBLE', 'INMUEBLES', 'TERRENO', 'TERRENOS', 'CASA', 'CASAS', 'DEPARTAMENTO', 'DEPARTAMENTOS',
    'CONSTRUCCIÓN', 'CONSTRUCCIONES', 'CONSTRUIR', 'CONSTRUYENDO', 'CONSTRUYÓ', 'CONSTRUIDO', 'CONSTRUIDA', 'CONSTRUIDOS', 'CONSTRUIDAS',
    'AMPARO POSESORIO', 'POSESIÓN', 'POSESION', 'POSESIÓN', 'POSESIONES', 'POSESIÓNES', 'POSESIÓNES', 'POSESIONES', 'POSESIÓNES', 'POSESIÓNES',
    'Dominio', 'DOMINIO PLENO', 'DOMINIO ÚTIL', 'DOMINIO DIRECTO', 'DOMINIO ÚTIL', 'DOMINIO DIRECTO', 'DOMINIO PLENO', 'DOMINIO ÚTIL', 'DOMINIO DIRECTO',
    'HIPOTECA', 'HIPOTECAS', 'HIPOTECARIO', 'HIPOTECARIOS', 'HIPOTECARIA', 'HIPOTECARIAS', 'HIPOTECADO', 'HIPOTECADA', 'HIPOTECADOS', 'HIPOTECADAS',

],

'transito' : [
    'TRÁNSITO', 'TRÁFICO', 'TRÁNSITO', 'TRÁFICO', 'VEHÍCULO', 'VEHÍCULOS', 'AUTOMÓVIL', 'AUTOMÓVILES', 'CAMIÓN', 'CAMIONES', 'MOTOCICLETA', 'MOTOCICLETAS',
    'CONDUCCIÓN', 'CONDUCCION', 'CONDUCCIÓN', 'TRANSPORTE' 
]


}

categories_delitos = {}
remaing_delitos = unique_delitos
for key, value in categories.items():
    filtered_delitos, remaing_delitos = filter_delitos(remaing_delitos, value)
    categories_delitos = {**categories_delitos, **{key: filtered_delitos}}
    #print(key, filtered_delitos)
    print(key, len(filtered_delitos))
    
print('remaing_delitos', remaing_delitos)
print('remaing_delitos', len(remaing_delitos))

        

alimentos 13
familiares 72
penales 42
estafa 15
cobros 40
propiedades 12
transito 21
remaing_delitos ['DERECHOS FUNDAMENTALES DEL CONSUMIDOR', 'ARCHIVO DE LA INVESTIGACIÓN PREVIA', 'REINVINDICACIÓN', 'RESOLUCIÓN DE CONTRATO', 'TERMINACIÓN DE CONTRATO', 'CONTRATO DE MUTUO O PRESTAMO', 'INCUMPLIMIENTO DE DECISIONES LEGÍTIMAS DE AUTORIDAD COMPETENTE', 'FACTURAS O DOCUMENTOS ART. 356 NUM.2', 'FACTURA O PRIMA DE SEGUROS', 'PARTICION NO VOLUNTARIA', 'APLICACIÓN DEL PRINCIPIO DE OPORTUNIDAD', 'NULIDAD DEL ACTA DE MEDIACIÓN', 'RECONOCIMIENTO Y HOMOLOGACIÓN DE SENTENCIA EXTRANJERA', 'EJECUCIÓN DE LAUDO ARBITRAL', 'INVENTARIO DE BIENES SUSCESORIOS', 'EJECUCIÓN DE SENTENCIA', 'INDEMNIZACIÓN POR DESPIDO INTEMPESTIVO', '309 USURA', 'USURPACIÓN', 'CONCURSO DE ACREEDORES', 'DOCUMENTOS ART. 356 NUM.1', 'OTROS', 'RECONOCIMIENTO DE DOCUMENTO PRIVADO', 'ACCIONES COLUSORIAS', 'DAÑO MORAL', 'DEPRECATORIO', 'DEFRAUDACIÓN TRIBUTARIA', 'LAS ORDENADAS POR LEY ', 'PARTICIÓN DE BIENES SUCESORIOS', 'EXÁMEN DE ADN

In [25]:
a

Cédula
9.284309e+08    99
1.400412e+09    88
7.046659e+08    87
1.400513e+09    86
1.400286e+09    86
                ..
9.422024e+08     1
9.422002e+08     1
9.422000e+08     1
9.421969e+08     1
1.104264e+09     1
Name: count, Length: 512903, dtype: int64


In [26]:
# add the cedula_counts back to the original data frame
df = pd.merge(df, cedula_counts, left_on='Cédula', right_index=True, how='left')

print(df.head())

        Cédula        id_Juicio                                  Delito  \
0  100004407.0  03204201700015               COBRO DE PAGARÉ A LA ORDEN   
1  100007269.0  01204201905175                                ALIMENTOS   
2  100007269.0  01204201903452                                ALIMENTOS   
3  100007269.0  01204201900368   INVESTIGACIÓN DE PATERNIDAD/MATERNIDAD   
4  100013465.0  01371202300415                PAGO DE HABERES LABORALES   

         Fecha_de_Ingreso        mes     año                 Estado  \
0 2017-01-09 13:23:24.813    January  2017.0              No activo   
1 2019-09-09 15:50:33.447  September  2019.0  No se pudo determinar   
2 2019-06-12 13:19:18.733       June  2019.0              No activo   
3 2019-01-18 21:50:53.770    January  2019.0              No activo   
4 2023-12-05 17:28:11.537   December  2023.0                 Activo   

  Monto_por_juicio  count  
0              NaN    1.0  
1              NaN    3.0  
2              NaN    3.0  
3         